In [1]:
library(lmtp)
library(twang)

Major changes in lmtp 0.9.0. Consult NEWS.md for more information.

To reproduce results from prior versions of the twang package, please see the version="legacy" option described in the documentation.



In [2]:
baseline <- c("W_1", "W_2")
trt <- c("A_1", "A_2")
time_vary <- list(c("L_11", "L_12"), c("L_21", "L_22"))
create_node_list(trt = trt, baseline = baseline, time_vary = time_vary, tau = 2)

$trt
$trt[[1]]
[1] "W_1"  "W_2"  "L_11" "L_12" "A_1" 

$trt[[2]]
[1] "W_1"  "W_2"  "L_11" "L_12" "L_21" "L_22" "A_1"  "A_2" 


$outcome
$outcome[[1]]
[1] "W_1"  "W_2"  "L_11" "L_12" "A_1" 

$outcome[[2]]
[1] "W_1"  "W_2"  "L_11" "L_12" "A_1"  "L_21" "L_22" "A_2"

In [3]:
shift <- function(data, trt) {
    (data[[trt]] - 1) * (data[[trt]] - 1 >= 1) + data[[trt]] * (data[[trt]] - 1 <
        1)
}

In [4]:
A <- c("A_1", "A_2", "A_3", "A_4")
L <- list(c("L_1"), c("L_2"), c("L_3"), c("L_4"))
lmtp_sdr(sim_t4, A, "Y", time_vary = L, k = 0, shift = shift, folds = 5)

Loading required package: nnls

LMTP Estimator: SDR

Trt. Policy: (shift)

Population intervention effect

Estimate: 0.2634

Std. error: 0.0181

95% CI: (0.228, 0.2988)



In [5]:
data(iptwExWide)

A <- paste0("tx", 1:3)
W <- c("gender", "age")
L <- list(c("use0"), c("use1"), c("use2"))
lmtp_tmle(iptwExWide, A, "outcome", W, L, shift = static_binary_on, outcome_type = "continuous",
    folds = 2, .SL_folds = 2)

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misl

In [6]:
A <- c("A_1", "A_2", "A_3", "A_4")
L <- list(c("L_1"), c("L_2"), c("L_3"), c("L_4"))
shift <- function(data, trt) {
  (data[[trt]] - 1) * (data[[trt]] - 1 >= 1) + data[[trt]] * (data[[trt]] - 1 < 1)
}

# creating a dynamic mtp that applies the shift function 
# but also depends on history and the current time
dynamic_mtp <- function(data, trt) {
  if (trt == "A_1") {
    # if its the first time point, follow the same mtp as before
    shift(data, trt)
  } else {
    # otherwise check if the time varying covariate equals 1
    ifelse(data[[sub("A", "L", trt)]] == 1, 
           shift(data, trt), # if yes continue with the policy
           data[[trt]])      # otherwise do nothing
  }
}

lmtp_tmle(sim_t4, A, "Y", time_vary = L, k = 0, 
          shift = dynamic_mtp, folds = 2, .SL_folds = 2)

LMTP Estimator: TMLE

Trt. Policy: (dynamic_mtp)

Population intervention effect

Estimate: 0.2347

Std. error: 0.0113

95% CI: (0.2126, 0.2568)



In [7]:
A <- paste0("tx", 1:3)
W <- c("gender", "age")
L <- list(c("use0"), c("use1"), c("use2"))
lrnrs <- c("SL.glm", "SL.ranger", "SL.glm.interaction")
lmtp_tmle(iptwExWide, A, "outcome", W, L, shift = static_binary_on, 
        outcome_type = "continuous", learners_trt = lrnrs, 
        learners_outcome = lrnrs, folds = 2, .SL_folds = 2)

Loading required namespace: ranger

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction f

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misl

In [8]:
head(sim_cens[sim_cens$C1 == 0, ])

,L1,A1,C1,L2,A2,C2,Y
,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>
48,1,3.260499,0,NA,NA,0,NA
58,1,1.109737,0,NA,NA,0,NA
67,1,1.376540,0,NA,NA,0,NA
112,1,3.832451,0,NA,NA,0,NA
114,1,1.241776,0,NA,NA,0,NA
115,1,2.390811,0,NA,NA,0,NA


In [9]:
A <- c("A1", "A2")
L <- list(c("L1"), c("L2"))
C <- c("C1", "C2")

lmtp_tmle(sim_cens, A, "Y", time_vary = L, cens = C,
          shift = function(data, trt) data[[trt]] + 0.5, 
          folds = 2, .SL_folds = 2)

LMTP Estimator: TMLE

Trt. Policy: (function(data, trt) data[[trt]] + 0.5)

Population intervention effect

Estimate: 0.899

Std. error: 0.0095

95% CI: (0.8804, 0.9176)



In [10]:
A <- c("A1", "A2")
L <- list(c("L1"), c("L2"))
C <- c("C1", "C2")

lmtp_sdr(sim_cens, A, "Y", time_vary = L, cens = C, 
         shift = NULL, folds = 2, .SL_folds = 2)

LMTP Estimator: SDR

Trt. Policy: (NULL)

Population intervention effect

Estimate: 0.7982

Std. error: 0.0129

95% CI: (0.7729, 0.8236)



In [11]:
A <- "trt"
Y <- paste0("Y.", 1:6)
C <- paste0("C.", 0:5)
W <- c("W1", "W2")

lmtp_tmle(sim_point_surv, A, Y, W, cens = C, shift = static_binary_on, 
          outcome_type = "survival", folds = 2, .SL_folds = 2)

Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misleading"
Warning message in predict.lm(object, newdata, se.fit, scale = 1, type = if (type == :
"prediction from a rank-deficient fit may be misl

In [12]:
W <- "L0.c"
L <- list(c("L0.a", "L0.b"), c("L1.a", "L1.b"))
A <- c("A0", "A1")
C <- c("C0", "C1")
Y <- c("Y1", "Y2")

lmtp_sdr(sim_timevary_surv, A, Y, W, L, C, outcome_type = "survival", 
         shift = static_binary_on, folds = 2, .SL_folds = 2)

Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: fitted probabilities numerically 0 or 1 occurred"
Warning message:
"glm.fit: algorithm did not converge"
Warning message:
"glm

In [13]:
A <- c("A1", "A2")
L <- list(c("L1"), c("L2"))
C <- c("C1", "C2")

fit_shift <- 
  lmtp_sdr(sim_cens, A, "Y", time_vary = L, cens = C, 
           shift = function(data, trt) data[[trt]] + 0.5, 
           folds = 2, .SL_folds = 2)

fit_noshift <- 
  lmtp_sdr(sim_cens, A, "Y", time_vary = L, cens = C,
           shift = NULL, folds = 2, .SL_folds = 2)

In [14]:
lmtp_contrast(fit_shift, ref = fit_noshift, type = "additive")

LMTP Contrast: additive

Null hypothesis: theta == 0




  
  theta shift   ref std.error conf.low conf.high p.value
1 0.101 0.899 0.798    0.0057   0.0894     0.112  <0.001

In [15]:
lmtp_contrast(fit_shift, ref = fit_noshift, type = "rr")

LMTP Contrast: relative risk

Null hypothesis: theta == 1




  
  theta shift   ref std.error conf.low conf.high p.value
1  1.13 0.899 0.798   0.00772     1.11      1.14  <0.001

In [16]:
tidy(fit_shift)

estimator,estimate,std.error,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
SDR,0.8985913,0.009827451,0.8793298,0.9178527
